In [1]:
import duckdb

# puzzle input goes here
input = """
"""

In [2]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT
            CAST(REGEXP_EXTRACT(i, 'Game ([0-9]+):', 1) AS INT) game_id,
            GENERATE_SUBSCRIPTS(STRING_SPLIT(i, ';'), 1) set_no,
            COALESCE(CAST(NULLIF(REGEXP_EXTRACT(UNNEST(STRING_SPLIT(i, ';')), ' ([0-9]+) red', 1), '') AS INT), 0) red,
            COALESCE(CAST(NULLIF(REGEXP_EXTRACT(UNNEST(STRING_SPLIT(i, ';')), ' ([0-9]+) green', 1), '') AS INT), 0) green,
            COALESCE(CAST(NULLIF(REGEXP_EXTRACT(UNNEST(STRING_SPLIT(i, ';')), ' ([0-9]+) blue', 1), '') AS INT), 0) blue
        FROM (
            SELECT UNNEST(STRING_SPLIT(input, '\n')) i
            FROM input
        )
    ), bag AS (
        SELECT 12 red, 13 green, 14 blue
    ), possible_games AS (
        SELECT
            game_id,
            SUM(CAST(t.red > bag.red AS INT)) red,
            SUM(CAST(t.green > bag.green AS INT)) green,
            SUM(CAST(t.blue > bag.blue AS INT)) blue
        FROM t, bag
        GROUP BY game_id
        HAVING SUM(CAST(t.red > bag.red AS INT)) = 0
        AND SUM(CAST(t.green > bag.green AS INT)) = 0
        AND SUM(CAST(t.blue > bag.blue AS INT)) = 0
    )
    SELECT SUM(game_id) ans
    FROM possible_games
""" % input)

┌────────┐
│  ans   │
│ int128 │
├────────┤
│   2776 │
└────────┘

In [3]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT
            CAST(REGEXP_EXTRACT(i, 'Game ([0-9]+):', 1) AS INT) game_id,
            GENERATE_SUBSCRIPTS(STRING_SPLIT(i, ';'), 1) set_no,
            COALESCE(CAST(NULLIF(REGEXP_EXTRACT(UNNEST(STRING_SPLIT(i, ';')), ' ([0-9]+) red', 1), '') AS INT), 0) red,
            COALESCE(CAST(NULLIF(REGEXP_EXTRACT(UNNEST(STRING_SPLIT(i, ';')), ' ([0-9]+) green', 1), '') AS INT), 0) green,
            COALESCE(CAST(NULLIF(REGEXP_EXTRACT(UNNEST(STRING_SPLIT(i, ';')), ' ([0-9]+) blue', 1), '') AS INT), 0) blue
        FROM (
            SELECT UNNEST(STRING_SPLIT(input, '\n')) i
            FROM input
        )
    ), bag AS (
        SELECT
            game_id,
            ARRAY_AGG(ROW(red, green, blue)),
            MAX(red) red,
            MAX(green) green,
            MAX(blue) blue
        FROM t
        GROUP BY game_id
    )
    SELECT SUM(red * green * blue) ans
    FROM bag
""" % input)

┌────────┐
│  ans   │
│ int128 │
├────────┤
│  68638 │
└────────┘